# dev-tooling.ipynb

Develop the API code to support tooling

## LiteLLM Original Example

In [46]:
import litellm
import importlib
print(importlib.metadata.version("litellm"))
import json
# set openai api key
import os
os.environ['OPENAI_API_KEY'] = os.environ['MY_OPENAI_API_KEY']
MODEL = "openai/gpt-4o"

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def test_parallel_function_call():
    try:
        # Step 1: send the conversation and available functions to the model
        messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                        },
                        "required": ["location"],
                    },
                },
            }
        ]
        response = litellm.completion(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
        )
        print("\nFirst LLM Response:\n", response)
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        print("\nLength of tool calls", len(tool_calls))

        # Step 2: check if the model wanted to call a function
        if tool_calls:
            # Step 3: call the function
            # Note: the JSON response may not always be valid; be sure to handle errors
            available_functions = {
                "get_current_weather": get_current_weather,
            }  # only one function in this example, but you can have multiple
            messages.append(response_message)  # extend conversation with assistant's reply

            # Step 4: send the info for each function call and function response to the model
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                )
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )  # extend conversation with function response
            second_response = litellm.completion(
                model=MODEL,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print("\nSecond LLM response:\n", second_response)
            return second_response
    except Exception as e:
      print(f"Error occurred: {e}")

test_ret1 = test_parallel_function_call()
test_ret1

1.66.0

First LLM Response:
 ModelResponse(id='chatcmpl-BPA3wPIN6GvkAkMIZMHu0EOoki25d', created=1745336580, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_a6889ffe71', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), id='call_9AjtVQmGvUqZe6YoChInWxuy', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Tokyo, Japan"}', name='get_current_weather'), id='call_q1li00gf6ci6ErIE5B2lCuli', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), id='call_C2aHqW8JiT6wBwkkELN3Rb3N', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=69, prompt_tokens=85, total_tokens=154, completion

/home/johann/software/anaconda/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...uy', 'type': 'function'}, input_type=dict])
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...li', 'type': 'function'}, input_type=dict])
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...3N', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(



Second LLM response:
 ModelResponse(id='chatcmpl-BPA3xcUSmxyJ6oZgwvu6nhVHRQOjm', created=1745336581, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_90122d973c', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Here's the current weather in the requested cities:\n\n- **San Francisco, CA**: 72°F\n- **Tokyo, Japan**: 10°C\n- **Paris, France**: 22°C", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=43, prompt_tokens=158, total_tokens=201, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')


ModelResponse(id='chatcmpl-BPA3xcUSmxyJ6oZgwvu6nhVHRQOjm', created=1745336581, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_90122d973c', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Here's the current weather in the requested cities:\n\n- **San Francisco, CA**: 72°F\n- **Tokyo, Japan**: 10°C\n- **Paris, France**: 22°C", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=43, prompt_tokens=158, total_tokens=201, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

## Own Lib

In [1]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
from llms_wrapper.llms import LLMS, toolnames2funcs, get_func_by_name
from llms_wrapper.config import update_llm_config

In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
config = update_llm_config(config)

In [3]:
llms = LLMS(config)

In [4]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

In [26]:
def query_names(where_clause: str) -> List[str]: 
    """
    Query the customer database and return a list of matching names. 

    This function queries the customer database using the conditions in the where clause and returns
    a list of matching customers. The where clause may use the DB fields "city", "company_name",
    "country" and "since_date" to limit the returned customer list. The where clause can also 
    be followed by a limit clause to limit the number of returned names. 

    :param where_clause: the string containing the where and optionally limit clauses in SQL query format
    :type where_clause: string
    :return: a list of matching customer names
    :rtype: array
    """
    return ["Monica Schmidt", "Harald Mueller"]


In [27]:
tmpfunc = get_func_by_name("query_names")
if callable(tmpfunc):
    print(f"Found {tmpfunc.__name__} as a function")
else:
    print(f"Found {tmpfunc} but not a function, type {type(tmpfunc)}")


Found query_names as a function


In [28]:
tmpfunc("city='Berlin' and company_name='Acme Corp' and since_date='2023-01-01' and limit=10")

['Monica Schmidt', 'Harald Mueller']

In [29]:
tools=llms.make_tooling(query_names)
tools

[{'type': 'function',
  'function': {'name': 'query_names',
   'description': 'Query the customer database and return a list of matching names. \n\nThis function queries the customer database using the conditions in the where clause and returns\na list of matching customers. The where clause may use the DB fields "city", "company_name",\n"country" and "since_date" to limit the returned customer list. The where clause can also \nbe followed by a limit clause to limit the number of returned names.',
   'parameters': {'type': 'object',
    'properties': {'where_clause': {'type': 'string',
      'description': 'the string containing the where and optionally limit clauses in SQL query format'}},
    'required': ['where_clause']}}}]

In [30]:
llms.supports_function_calling("openai/gpt-4o")

True

In [31]:
llms.supports_function_calling("openai/gpt-4o", parallel=True)

False

In [32]:
msgs = llms.make_messages("Give me the names of customers in New York which have been customers since 2023 or longer")
msgs

[{'content': 'Give me the names of customers in New York which have been customers since 2023 or longer',
  'role': 'user'}]

In [33]:
ret = llms.query("openai/gpt-4o", messages=msgs, tools=tools, return_cost=True)
ret["answer"], ret["error"]

/home/johann/software/anaconda/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...xq', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(


('The customers in New York who have been customers since 2023 or longer are:\n\n- Monica Schmidt\n- Harald Mueller',
 '')

In [13]:
ret

{'elapsed_time': 1.8553009033203125,
 'cost': 0.0015425,
 'n_completion_tokens': 63,
 'n_prompt_tokens': 365,
 'n_total_tokens': 428,
 'finish_reason': 'stop',
 'answer': 'Here are the names of customers in New York who have been customers since 2023 or longer:\n\n1. Monica Schmidt\n2. Harald Mueller',
 'error': '',
 'ok': True}